In [1]:
with open('clq5.txt', 'r', encoding='utf-8') as file:
    contents = file.read()
# print(contents)

In [2]:
hindi_vowels = ['अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ', 'अं', 'अः', 'अँ']
matra = ['ा', 'ि', 'ी', 'ु', 'ू', 'े', 'ो', 'ै', 'ौ', 'ृ', 'ॄ', 'ॉ', 'ं','्','ँ']
hindi_consonants = [
    'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ',
    'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न',
    'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श',
    'ष', 'स', 'ह', 'ळ', 'क्ष', 'ज्ञ'
]
mapping = {'ा': 'आ', 'ि': 'इ', 'ी': 'ई', 'ु': 'उ', 'ू': 'ऊ', 'ृ': 'ऋ', 'ॄ': 'ॠ', 'ॅ': 'ऍ', 'े': 'ए', 'ै': 'ऐ', 'ो': 'ओ', 'ौ': 'औ', 'ं': 'अं', 'ः': 'अः', 'ँ': 'अँ'}
def clean_token(tokens1):
    tokens = []
    for word in tokens1:
        s=""
        for c in word:
            if (c not in matra) and (c not in hindi_consonants) and (c not in hindi_vowels): continue
            else: s = s + c
        if(len(s)): tokens.append(s)
    return tokens

# TOKENIZATION MODELS:

In [3]:
import sentencepiece as spm

## UNIGRAM

In [4]:
corpus_file = "hi_100.txt"
model_prefix = "hindi_unigram"
vocab_size = 1000 
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix=model_prefix, vocab_size=vocab_size, model_type='unigram')
sp = spm.SentencePieceProcessor()
sp.load(f"{model_prefix}.model")
def unigram_model(sen):
    tokens1 = sp.encode_as_pieces(sen)
    return tokens1

## BPE MODEL (VOCAB_SIZE : 1000)

In [5]:
corpus_file = "hi_100.txt"
model_prefix = "hindi_bpe"
vocab_size = 1000 
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix=model_prefix, vocab_size=vocab_size, model_type='bpe')
sp_b1 = spm.SentencePieceProcessor()
sp_b1.load(f"{model_prefix}.model")
def bpe_1000(sen):
    tokens2 = sp_b1.encode_as_pieces(sen)
    return tokens2

## BPE MODEL (VOCAB_SIZE : 2000)

In [6]:
corpus_file = "hi_100.txt"
model_prefix = "hindi_bpe2"
vocab_size = 2000
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix=model_prefix, vocab_size=vocab_size, model_type='bpe')
sp_b1 = spm.SentencePieceProcessor()
sp_b1.load(f"{model_prefix}.model")
def bpe_2000(sen):
    tokens3 = sp_b1.encode_as_pieces(sen)
    return tokens3

## m-BERT  (MAX-LENGTH:1000)

In [7]:
from transformers import BertTokenizer
def mbert_m(sen):
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
    tokens4 = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sen, max_length=1000, truncation=True, padding='max_length')))
    return tokens4

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## m-BERT  (MAX-LENGTH:2000)

In [8]:
from transformers import BertTokenizer
def mbert_m2(sen):
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
    tokens5 = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sen, max_length=2000, truncation=True, padding='max_length')))
    return tokens5

## Indic-BERT (MAX-LENGTH:1000)

In [9]:
from transformers import AutoTokenizer
def indicbert(sen):
    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    tokens6 = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sen, max_length=1000, truncation=True)))
    return tokens6

## Indic-BERT (MAX-LENGTH:2000)

In [10]:
from transformers import AutoTokenizer
def indicbert2(sen):
    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    tokens7 = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sen, max_length=1000, truncation=True)))
    return tokens7

## WHITE SPACE TOKENIZER

In [11]:
def white_space(sen):
    tokens8 = contents.split()
    return tokens8

## FUNCTION FOR CALCULATION OF PRECISION, RECALL, F-SCORE:

In [12]:
def score_cal(ground_truth, tokenizer_output):
    true_positives = 0.0
    false_positives = 0.0
    false_negatives = 0.0
    for i in range(0, len(ground_truth)):
        true_positives += len(set(ground_truth[i]) & set(tokenizer_output[i]))
        false_positives += len(set(tokenizer_output[i]) - set(ground_truth[i]))
        false_negatives += len(set(ground_truth[i]) - set(tokenizer_output[i]))
    if (true_positives + false_positives) != 0: precision = true_positives / (true_positives + false_positives)
    else : precision = 0
    if (true_positives + false_negatives) != 0: recall = true_positives / (true_positives + false_negatives)
    else : recall = 0
    if (precision + recall) != 0: f_score = 2 * (precision * recall) / (precision + recall)
    else : f_score = 0
    return precision, recall, f_score

## SOLUTION OF WORD-GROUPS FROM QUES-3 : 

In [13]:
gt = [['नगर पालिका कर्मचारी संघ', 'हरियाणा के', 'आह्वïान पर', 'आज यहां', 'नगर निगम के कर्मचारियों ने', 'जिला प्रधान धर्मवीर सारसर की', 'अध्यक्षता में', 'मांगों के समर्थन', 'में भूख हड़ताल की'], 
['आपको', 'अलग-अलग परीक्षण करना', 'चाहिए', 'और', 'खोजशब्द शोधकर्ताओं को', 'अपनी सूची में', 'सबसे अधिक प्रासंगिक और लक्षित खोज शब्दों को', 'खोजने के लिए', 'प्रदान करना चाहिए'], 
['मोका कारवाईंतर्गंत', 'म्होरक्यासह', '97 गुंडांना', 'झटका'], 
['अमेज़न और फ्लिपकार्ट', 'को टक्कर देगा', 'ई-कॉमर्स पोर्टल GeM'],
['उम्मीद है', 'कि यह बिजली संयंत्र', '6 से 9 महीने में', 'लग जाएगा', 'और', 'कम से कम 15 साल तक', 'चलेगा'],
['घायल हुए', 'इस स्वर्ण व्यवसायी की', 'पहचान', 'जिला के सैदपुरा के', 'रहने वाले', '50 वर्षीय राज किशोर साह', 'के रूप में', 'की गयी है', 'जो', 'काजीमोहम्मद पुर थाना क्षेत्र के', 'सतपुरा इलाके के', 'निवासी', 'थे', 'और', 'घटना के वक्त', 'अपने छोटे बेटे के साथ', 'दुकान बंद कर घर वापस लौट रहे थे'], 
['सरकार ने', 'विभिन्न खेलों के', 'इन खिलाडिय़ों में से', 'चार को', 'सीधे पुलिस उपाधीक्षक (डीएसपी)', '18 को', 'निरीक्षक (इंस्पेक्टर)', 'तथा', '14 को', 'उप-निरीक्षक (सब-इंस्पेक्टर)', 'नियुक्त किया है'], ['दौर सराफा में', 'Gold', '40,080 रुपए प्रति 10 ग्राम', 'बिका'], 
['इस ट्रायल में', 'शामिल रहे', 'एक वरिष्ठ वैज्ञानिक डॉक्टर आरएस शर्मा', 'का कहना है', 'कि यह इंजेक्शन पूरी तरह तैयार है', 'केवल ड्रग कंट्रोलर से', 'इसकी अनुमति लिया जाना बाक़ी है'], 
['घर में', 'शुभ प्रसंग', 'का', 'आयोजन हो सकता है'], 
['हितग्राही को', 'यह जरूर', 'महसूस होना चाहिए', 'कि मुझे जो सहायता मिल रही है', 'वह खैरात नहीं है', 'बल्कि मेरी लियाकत की वजह से है', 'मेरी मेहनत के फलस्वरूप है'], ['आरबीआई ने', 'मामले की', 'जांच शुरू कर दी है'], 
['जिनका', 'जिक्र', 'अंग्रेजी के', 'शब्दकोश में', 'है ही नहीं'], 
['बेशक', 'प्रधानमंत्री डॉ. मनमोहन सिंह', 'यह मानें', 'कि इससे देश की बदनामी होती है'], 
['विद्यार्थियों को', 'कंप्यूटर की सॉफ्टवेयर व हार्डवेयर की हर प्रकार की', 'जानकारी होना आवश्यक है', 'और सीखने आए विद्यार्थियों ने', 'कहा', 'कि यह जानकारी उनके लिए अच्छी है'], ['बाद में', 'भास्कर के जीएम कमल शर्मा', 'वकील के साथ', 'थाने गए', 'और', 'पालना का', 'आश्वसन देकर', 'आए'], 
['बैठक के बाद से', 'मीडिया में', 'कयास लगाए जाने लगे थे', 'कि जम्मू-कश्मीर पर', 'केंद्र सरकार ने', 'कोई बड़ा फैसला', 'ले लिया है'], 
['नारायण साईं पर', 'जेल में रहते हुए', 'पुलिस कर्मचारी को', '13 करोड़ रुपए की', 'रिश्वत देने का', 'भी आरोप लगा था', 'लेकिन इस मामले में', 'नारायण साईं को', 'जमानत तो मिल चुकी है', 'लेकिन रेप के मामले में', 'अभी भी', 'कोर्ट में', 'सुनवाई चल रही है'], 
['इस बार', '10वीं में', '34,04,471', 'और', '12वीं में', '26,24,681', 'यानि', 'कुल', '60,29,152 लाख', 'रेगुलर और प्राइवेट विद्यार्थियों', 'ने', 'भाग लिया था'], ['और जब', 'वो पानी से', 'निकल कर', 'ऊपर आया', 'तो फ़ौरन', 'उसने', 'आसमान को', 'खुलते और रूह को', 'कबूतर की तरह', 'अपने ऊपर आते', 'देखा'], ['उस ताम्बे की परत पे', 'जमे परतों के', 'जो रंग हैं'], 
['मार्सिलिया (Marsilea) तथा सिरेटोप्टेरिस (Ceratopteris)', 'जैसे', 'टेरिडोफाइट्स का', 'उपयोग', 'सब्जी के रूप में', 'होता है'], ['लेकिन अब', 'उनके डांस का', 'वीडियो', 'वायरल होने के बाद', 'तो उनकी लॉटरी लग गई है'], 
['इस दौरान', 'मारा मार्टिन ने', 'स्वीमशूट पहना हुआ है', 'वीडियो में', 'साफ नजर आ रहा है', 'कि जब मारा मार्टिन', 'रैंफ वॉक कर रही हैं', 'तो वहां मौजूद लोग', 'जमकर उनके लिए', 'तालियां बजा रहे हैं'], 
['तीन बच्चों की', 'मां', 'बियोंसे का', 'मानना है', 'कि स्तनपान', 'बच्चे को', 'जन्म देने के बाद', 'वजन घटाने का', 'सबसे कारगर और स्वस्थ तरीका है']]

## TOKENIZE ON UNIGRAM:

In [14]:
lines = contents.split("\n")
lst_lines = []
for sentence in lines:
    token = unigram_model(sentence)
    ctoken = clean_token(token)
    lst_lines.append(ctoken)
# print(lst_lines)

In [15]:
precision, recall, f_score = score_cal(gt,  lst_lines)
print("Unigram Tokenizer Metrics:")
print("Precision: ",precision)
print("Recall: ",recall)
print("F-score: ",f_score)

Unigram Tokenizer Metrics:
Precision:  0.021170610211706103
Recall:  0.09239130434782608
F-score:  0.034447821681864235


## TOKENIZE ON BPE FOR VOCAB_SIZE 1000:

In [16]:
lst_linesb = []
for sentence in lines:
    token = bpe_1000(sentence)
    ctoken = clean_token(token)
    lst_linesb.append(ctoken)
# print(lst_linesb)

In [17]:
precision, recall, f_score = score_cal(gt,  lst_linesb)
print("Bigram - 1000 Tokenizer Metrics:")
print("Precision: ",precision)
print("Recall: ",recall)
print("F-score: ",f_score)

Bigram - 1000 Tokenizer Metrics:
Precision:  0.02981029810298103
Recall:  0.11956521739130435
F-score:  0.047722342733188726


## TOKENIZE ON BPE FOR VOCAB_SIZE 2000:

In [18]:
lst_linesb2 = []
for sentence in lines:
    token = bpe_2000(sentence)
    ctoken = clean_token(token)
    lst_linesb2.append(ctoken)
# print(lst_linesb2)

In [19]:
precision, recall, f_score = score_cal(gt,  lst_linesb2)
print("Bigram - 2000 Tokenizer Metrics:")
print("Precision: ",precision)
print("Recall: ",recall)
print("F-score: ",f_score)

Bigram - 2000 Tokenizer Metrics:
Precision:  0.02981029810298103
Recall:  0.11956521739130435
F-score:  0.047722342733188726


## TOKENIZE ON mBERT - 1000:

In [20]:
lst_linesm = []
for sentence in lines:
    token = mbert_m(sentence)
    ctoken = clean_token(token)
    lst_linesm.append(ctoken)
# print(lst_linesm)

In [21]:
precision, recall, f_score = score_cal(gt,  lst_linesm)
print("mBERT Tokenizer Metrics:")
print("Precision: ",precision)
print("Recall: ",recall)
print("F-score: ",f_score)

mBERT Tokenizer Metrics:
Precision:  0.020531400966183576
Recall:  0.09239130434782608
F-score:  0.03359683794466403


## TOKENIZE ON mBERT - 2000:

In [22]:
lst_linesm2 = []
for sentence in lines:
    token = mbert_m2(sentence)
    ctoken = clean_token(token)
    lst_linesm2.append(ctoken)
# print(lst_linesm2)

In [23]:
precision, recall, f_score = score_cal(gt,  lst_linesm2)
print("mBERT Tokenizer Metrics:")
print("Precision: ",precision)
print("Recall: ",recall)
print("F-score: ",f_score)

mBERT Tokenizer Metrics:
Precision:  0.020531400966183576
Recall:  0.09239130434782608
F-score:  0.03359683794466403


## TOKENIZE ON INDIC-BERT - 1000:

In [24]:
lst_linesi = []
for sentence in lines:
    token = indicbert(sentence)
    ctoken = clean_token(token)
    lst_linesi.append(ctoken)
# print(lst_linesi)

In [25]:
precision, recall, f_score = score_cal(gt,  lst_linesi)
print("Indic BERT Tokenizer Metrics:")
print("Precision: ",precision)
print("Recall: ",recall)
print("F-score: ",f_score)

Indic BERT Tokenizer Metrics:
Precision:  0.009917355371900827
Recall:  0.03260869565217391
F-score:  0.015209125475285173


## TOKENIZE ON INDIC-BERT - 2000:

In [26]:
lst_linesi2 = []
for sentence in lines:
    token = indicbert2(sentence)
    ctoken = clean_token(token)
    lst_linesi2.append(ctoken)
# print(lst_linesi2)

In [27]:
precision, recall, f_score = score_cal(gt,  lst_linesi)
print("Indic BERT Tokenizer Metrics:")
print("Precision: ",precision)
print("Recall: ",recall)
print("F-score: ",f_score)

Indic BERT Tokenizer Metrics:
Precision:  0.009917355371900827
Recall:  0.03260869565217391
F-score:  0.015209125475285173


## TOKENIZE ON WHITE-SPACE TOKENIZER

In [28]:
lines = contents.split("\n")
lst_linesw = []
for sentence in lines:
    token = sentence.split(" ")
    lst_linesw.append(token)
print(lst_linesw)

[['नगर', 'पालिका', 'कर्मचारी', 'संघ', 'हरियाणा', 'के', 'आह्वïान', 'पर', 'आज', 'यहां', 'नगर', 'निगम', 'के', 'कर्मचारियों', 'ने', 'जिला', 'प्रधान', 'धर्मवीर', 'सारसर', 'की', 'अध्यक्षता', 'में', 'मांगों', 'के', 'समर्थन', 'में', 'भूख', 'हड़ताल', 'की।'], ['आपको', 'अलग-अलग', 'परीक्षण', 'करना', 'चाहिए', 'और', 'खोजशब्द', 'शोधकर्ताओं', 'को', 'अपनी', 'सूची', 'में', 'सबसे', 'अधिक', 'प्रासंगिक', 'और', 'लक्षित', 'खोज', 'शब्दों', 'को', 'खोजने', 'के', 'लिए', 'प्रदान', 'करना', 'चाहिए।', ''], ['मोका', 'कारवाईंतर्गंत\xa0म्होरक्यासह', '97', 'गुंडांना', 'झटका'], ['अमेज़न', 'और', 'फ्लिपकार्ट', 'को', 'टक्कर', 'देगा', 'ई-कॉमर्स', 'पोर्टल', 'GeM'], ['उम्मीद', 'है', 'कि', 'यह', 'बिजली', 'संयंत्र', '6', 'से', '9', 'महीने', 'में', 'लग', 'जाएगा', 'और', 'कम', 'से', 'कम', '15', 'साल', 'तक', 'चलेगा।'], ['घायल', 'हुए', 'इस', 'स्वर्ण', 'व्यवसायी', 'की', 'पहचान', 'जिला', 'के', 'सैदपुरा', 'के', 'रहने', 'वाले', '50', 'वर्षीय', 'राज', 'किशोर', 'साह', 'के', 'रूप', 'में', 'की', 'गयी', 'है', 'जो', 'काजीमोहम्मद', 'पुर', 'थाना'

In [29]:
precision, recall, f_score = score_cal(gt,  lst_linesw)
print("White Space Tokenizer Metrics:")
print("Precision: ",precision)
print("Recall: ",recall)
print("F-score: ",f_score)

White Space Tokenizer Metrics:
Precision:  0.05731225296442688
Recall:  0.15760869565217392
F-score:  0.08405797101449276
